Stock trading data: date, code, open, high, low, close, preclose, volume, amount, adjustflag, turn,
     tradestatus, pctChg, peTTM, pbMRQ, psTTM, pcfNcfTTM, isST

Data source: baostock
    
Sample frequency: daily
    
Company scope: all A-share listed companies in China, including delisted companies


In [1]:
pip install baostock

     |████████████████████████████████| 61kB 218kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install os

  ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
Note: you may need to restart the kernel to use updated packages.


In [3]:
today = "2021-10-20"  

import pandas as pd
import tushare as ts
import os
import numpy as np
from tqdm import tqdm
import time
import pickle
import baostock as bs
lg = bs.login()
rs = bs.query_stock_basic()
rs = rs.get_data()
code_list = list(rs[rs["type"]=="1"]["code"])



login success!


In [4]:
#获取数据
class download_trade_data(object):
    def __init__(self,code,today,file):
        self.code = code
        self.today = today
        self.file = file
        
    def get_data(self,rs):
        data_list = []
        while (rs.error_code == '0') & rs.next():
            # 获取一条记录，将记录合并在一起
            data_list.append(rs.get_row_data())
        result = pd.DataFrame(data_list, columns=rs.fields)
        return result
    
    def get_ipodate(self):
        try:
            rs = bs.query_stock_basic()
            result = self.get_data(rs)
            date = result["ipoDate"].values[0]
        except:
            date = ""
        return date
    
    def get_history_data(self):
        date = self.get_ipodate()
        rs = bs.query_history_k_data_plus(self.code,
                                          "date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,peTTM,pbMRQ,psTTM,pcfNcfTTM,isST",
                                          start_date=date, 
                                          end_date=self.today,
                                          frequency="d", 
                                          adjustflag="3")  #1后复权数据 2前复权 3不复权
        data = self.get_data(rs)
        
        adclose = bs.query_history_k_data_plus(self.code,
                                  "close",
                                  start_date=date, 
                                  end_date=self.today,
                                  frequency="d", 
                                  adjustflag="1")  #1后复权数据 2前复权 3不复权
        addata = self.get_data(adclose)
        addata.columns = ["adclose"]
        data = pd.concat([data,addata],axis=1)
         
        data.to_csv(self.file+self.code+".csv")
        return
    
    @classmethod
    def check(cls,path):
        data_list = os.listdir(path)
        data_list = [x.split(".csv")[0] for x in data_list]
        r = list(set(code_list) - set(data_list))
        pr = "/".join(r)
        return r

In [ ]:
file="/Users/cr/Downloads/undergraduate time/大四上/FIN 4998/paper/code/data & data code/"

if __name__ == '__main__':
    for code in tqdm(code_list):
        down = download_trade_data(code,today,file)
        down.get_history_data()
#cost about 20 mins

 62%|██████▏   | 2884/4672 [2:15:18<39:34,  1.33s/it]     

In [6]:
bs.logout()
r = download_trade_data.check(file)
r

logout success!


[]

In [7]:
oslist = os.listdir(file)
data = []
for i in tqdm(oslist):
    s = pd.read_csv(file+i,index_col=0)
    temp = s.values.tolist()
    col = s.columns
    data.extend(temp)
data = pd.DataFrame(data,columns=col)

100%|██████████| 4649/4649 [00:57<00:00, 80.78it/s] 


In [9]:
def datefun(x):
    return ''.join(x.split('-'))

def codefun(x):
    f,b = x.split('.')
    return b + '.' + f.upper()

data['date'] = data['date'].apply(lambda x:datefun(x))
data['code'] = data['code'].apply(lambda x:codefun(x))
data = data.set_index(['code','date'])

In [60]:
import pickle
spath = '/Users/cr/Downloads/undergraduate time/大四上/FIN 4998/paper/code/data & data code/'
pickle.dump(data,open(spath+'day','wb'))